## 固有表現認識

### 固有表現認識とは？

In [5]:
# !pip install transformers[ja,sentencepice]

In [6]:
# !pip install spacy-alignments seqeval

#### データセットのダウンロード

In [7]:
from datasets import load_dataset

# データセットを読み込む
dataset = load_dataset("llm-book/ner-wikipedia-dataset")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration default


Dataset ner-wikipedia-dataset downloaded and prepared to /root/.cache/huggingface/datasets/llm-book___ner-wikipedia-dataset/default/0.0.0/184bcf9be66116e777f2f534436226d47348676c93ba20cca58933f1b2b3b782. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 709.18it/s]


In [8]:
# データセットの形式と事例数を確認する
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['curid', 'text', 'entities'],
        num_rows: 4274
    })
    validation: Dataset({
        features: ['curid', 'text', 'entities'],
        num_rows: 534
    })
    test: Dataset({
        features: ['curid', 'text', 'entities'],
        num_rows: 535
    })
})


In [9]:
from pprint import pprint

# 訓練セットの最初の二つの事例を表示する
pprint(list(dataset["train"])[:2])

[{'curid': '3638038',
  'entities': [{'name': 'さくら学院', 'span': [0, 5], 'type': 'その他の組織名'},
               {'name': 'Ciao Smiles', 'span': [6, 17], 'type': 'その他の組織名'}],
  'text': 'さくら学院、Ciao Smilesのメンバー。'},
 {'curid': '1729527',
  'entities': [{'name': 'レクレアティーボ・ウェルバ', 'span': [17, 30], 'type': 'その他の組織名'},
               {'name': 'プリメーラ・ディビシオン', 'span': [32, 44], 'type': 'その他の組織名'}],
  'text': '2008年10月5日、アウェーでのレクレアティーボ・ウェルバ戦でプリメーラ・ディビシオンでの初得点を決めた。'}]


#### データセットの分析

In [10]:
from collections import Counter
import pandas as pd
from datasets import Dataset

def count_label_occurrences(dataset: Dataset) -> dict[str, int]:
    """固有表現タイプの出現回数をカウント"""
    # 各事例から固有表現タイプを抽出したlistを作成する
    entities = [
        e["type"] for data in dataset for e in data["entities"]
    ]
    
    # ラベルの表現回数が多い順に並べる
    label_counts = dict(Counter(entities).most_common())
    return label_counts
    
label_counts_dict = {}
for split in dataset: # 各分割セットを処理する
    label_counts_dict[split] = count_label_occurrences(dataset[split])
# DataFrame形式で表示する
df = pd.DataFrame(label_counts_dict)
df.loc["合計"] = df.sum()
display(df)

,train,validation,test
人名,2394,299,287
法人名,2006,231,248
地名,1769,184,204
政治的組織名,953,121,106
製品名,934,123,158
施設名,868,103,137
その他の組織名,852,99,100
イベント名,831,85,93
合計,10607,1245,1333


#### スパンの重なる固有表現の存在を判定

In [11]:
def has_overlap(spans: list[tuple[int, int]]) -> int:
    """スパンの重なる固有表現の存在を判定"""
    sorted_spans = sorted(spans, key=lambda x: x[0])
    for i in range(1, len(sorted_spans)):
        # 前のスパンの終了位置が現在のスパンの開始位置より大きい場合、
        # 重なっているとする
        if sorted_spans[i - 1][1] > sorted_spans[i][0]:
            return 1
    return 0
    
# 各分割セットでスパンの重なる固有表現がある事例数を数える
overlap_count = 0
for split in dataset: # 各分割セットを処理する
    for data in dataset[split]: # 各事例を処理する
        if data["entities"]: # 固有表現の存在しない事例はスキップ
            # スパンのみのlistを作成する
            spans = [e["span"] for e in data["entities"]]
            overlap_count += has_overlap(spans)
    print(f"{split}におけるスパンが重複する事例数:{overlap_count}")

trainにおけるスパンが重複する事例数:0
validationにおけるスパンが重複する事例数:0
testにおけるスパンが重複する事例数:0


#### 前処理

#### テキストの正規化

In [12]:
from unicodedata import normalize

# テキストに対してUnicode正規化を行う
text =  "ＡＢＣABCａｂｃabcｱｲｳアイウ①②③123"
normalized_text = normalize("NFKC", text)
print(f"正規化前: {text}")
print(f"正規化後: {normalized_text}")

正規化前: ＡＢＣABCａｂｃabcｱｲｳアイウ①②③123
正規化後: ABCABCabcabcアイウアイウ123123


In [13]:
# 文字列の長さが変わる場合ある
text = "㈱、3㌕、10℃"
normalized_text = normalize("NFKC", text)
print(f"正規化前: {text}")
print(f"正規化後: {normalized_text}")

正規化前: ㈱、3㌕、10℃
正規化後: (株)、3キログラム、10°C


In [14]:
from unicodedata import normalize, is_normalized

count = 0
for split in dataset: # 各分割セットを処理する
    for data in dataset[split]: # 各事例を処理する
        # テキストが正規化されていない事例をカウントする
        if not is_normalized("NFKC", data["text"]):
            count += 1
print(f"正規化されていない事例数: {count}")

正規化されていない事例数: 0


#### テキストのトークナイゼーション

In [15]:
from transformers import AutoTokenizer

# トークナイザを読み込み
model_name = "tohoku-nlp/bert-base-japanese-v3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# トークナイゼーションを行う
subwords = "/".join(tokenizer.tokenize(dataset["train"][0]["text"]))
characters = "/".join(dataset["train"][0]["text"])
print(f"サブワード単位: {subwords}")
print(f"文単位: {characters}")

サブワード単位: さくら/学院/、/C/##ia/##o/Sm/##ile/##s/の/メンバー/。
文単位: さ/く/ら/学/院/、/C/i/a/o/ /S/m/i/l/e/s/の/メ/ン/バ/ー/。


#### 文字列とトークン列のアライメント

In [16]:
text = "さくら学院"

In [17]:
from spacy_alignments.tokenizations import get_alignments

# 文字列のlistを獲得する
characters = list(text)
# テキストを特殊トークンを含めたトークンのlistに変換する
tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(text))
# 文字のlistとトークンのlistのアライメントをとる
char_to_token_indices, token_to_char_indices = get_alignments(characters, tokens)
print(f"文字のlist: {characters}")
print(f"トークンのlist: {tokens}")
print(f"文字に対するトークンの位置: {char_to_token_indices}")
print(f"トークンに対する文字の位置: {token_to_char_indices}")

文字のlist: ['さ', 'く', 'ら', '学', '院']
トークンのlist: ['[CLS]', 'さくら', '学院', '[SEP]']
文字に対するトークンの位置: [[1], [1], [1], [2], [2]]
トークンに対する文字の位置: [[], [0, 1, 2], [3, 4], []]


#### 系列ラベリングのためのラベル作成

In [18]:
text = "大谷翔平は岩手県水沢市出身"
entities = [
    {"name": "大谷翔平", "span": [0,4], "type": "人名"},
    {"name": "岩手県水沢市", "span": [5,11], "type": "地名"},
]

In [19]:
from transformers import PreTrainedTokenizer

def output_tokens_and_labels(
    text: str,
    entities: list[dict[str, list[int] | str]],
    tokenizer: PreTrainedTokenizer,
) -> tuple[list[str], list[str]]:
    """トークンのlistとラベルのlistを出力"""
    # 文字列のlistとトークンのlistのアライメントをとる
    characters = list(text)
    tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(text))
    char_to_token_indices, _ = get_alignments(characters, tokens)
    
    # "O"のラベルで初期化したラベルのlistを作成する
    labels = ["O"] * len(tokens)
    for entity in entities: # 各固有表現で処理する
        entity_span, entity_type = entity["span"], entity["type"]
        start = char_to_token_indices[entity_span[0]][0]
        end = char_to_token_indices[entity_span[1]-1][0]
        # 固有表現の開始トークンの位置に"B-"のラベルを設定する
        labels[start] = f"B-{entity_type}"
        # 固有表現の開始トークン以外の位置に"I-"のラベルを設定する
        for idx in range(start + 1, end + 1):
            labels[idx] = f"I-{entity_type}"
    # 特殊トークンの位置にはラベルを設定しない
    labels[0] = "-" # 開始
    labels[-1] = "-" # 終了
    return tokens, labels

# トークンとラベルのlistを出力する
tokens, labels = output_tokens_and_labels(text, entities, tokenizer)
# DataFrameの形式で表示する
df = pd.DataFrame({"トークン列": tokens, "ラベル列": labels})
df.index.name = "位置"
display(df.T)

位置,0,1,2,3,4,5,6,7,8,9,10
トークン列,[CLS],大谷,翔,##平,は,岩手,県,水沢,市,出身,[SEP]
ラベル列,-,B-人名,I-人名,I-人名,O,B-地名,I-地名,I-地名,I-地名,O,-


#### 評価指標

#### seqevalライブラリを用いた評価スコアの算出

In [20]:
from typing import Any
from seqeval.metrics import classification_report

def create_character_labels(
    text: str, entities: list[dict[str, list[int] | str]]
) -> list[str]:
    """文字ベースでラベルのlistを作成"""
    # "O"のラベルで初期化したラベルのlistを作成する
    labels = ["O"] * len(text)
    for entity in entities: # 各固有表現を処理する
        entity_span, entity_type = entity["span"], entity["type"]
        # 固有表現の開始文字の位置に"B-"のラベルを設定する
        labels[entity_span[0]] = f"B-{entity_type}"
        # 固有表現の開始文字以外の位置に"I-"ラベルを設定する
        for i in range(entity_span[0] + 1, entity_span[1]):
            labels[i] = f"I-{entity_type}"
    return labels
    
def convert_results_to_labels(
    results: list[dict[str, Any]]
) -> tuple[list[list[str]], list[list[str]]]:
    """正解データと予測データのラベルのlistを作成"""
    true_labels, pred_labels = [], []
    for result in results: # 各事例を処理する
        # 文字ベースでラベルのリストを作成してlistに加える
        true_labels.append(
            create_character_labels(result["text"], result["entities"])
        )
        pred_labels.append(
            create_character_labels(result["text"], result["pred_entities"])
        )
    return true_labels, pred_labels

In [21]:
results = [
    {
        "text": "大谷翔平は岩手県水沢市出身",
        "entities": [
            {"name": "大谷翔平", "span": [0,4], "type": "人名"},
            {"name": "岩手県水沢市", "span": [5, 11], "type": "地名"}
        ],
        "pred_entities": [
            {"name": "大谷翔平", "span": [0,4], "type": "人名"},
            {"name": "岩手県", "span": [5,8], "type": "地名"},
            {"name": "水沢市", "span": [8,11], "type": "施設名"}
        ],
    }
]

# 正解データと予測データのラベルのlistを作成
true_labels, pred_labels = convert_results_to_labels(results)
# 評価結果を取得して表示
print(classification_report(true_labels, pred_labels))

              precision    recall  f1-score   support

          人名       1.00      1.00      1.00         1
          地名       0.00      0.00      0.00         1
         施設名       0.00      0.00      0.00         0

   micro avg       0.33      0.50      0.40         2
   macro avg       0.33      0.33      0.33         2
weighted avg       0.50      0.50      0.50         2



/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
from seqeval.metrics import f1_score, precision_score, recall_score

def compute_scores(
    true_labels: list[list[str]], pred_labels: list[list[str]], average: str
) -> dict[str, float]:
    """適合率、再現率、F値を算出"""
    scores = {
        "precision": precision_score(true_labels, pred_labels, average=average),
        "recall": recall_score(true_labels, pred_labels, average=average),
        "f1-score": f1_score(true_labels, pred_labels, average=average),
    }
    return scores

# 適合率、再現率、F値のマイクロ平均を算出する
print(compute_scores(true_labels, pred_labels, "micro"))
# 適合率、再現率、F値のマクロ平均を算出する
print(compute_scores(true_labels, pred_labels, "macro"))

{'precision': 0.3333333333333333, 'recall': 0.5, 'f1-score': 0.4}
{'precision': 0.3333333333333333, 'recall': 0.3333333333333333, 'f1-score': 0.3333333333333333}


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### 固有表現認識モデルの実装

#### BERTのファインチューニング

In [23]:
# ラベルとIDを対応付けるdictの作成
import torch

def create_label2id(
    entities_list: list[list[dict[str, str | int]]]
) -> dict[str, int]:
    """ラベルとIDを紐づけるdictを作成"""
    # "O"のIDには0を割り当てる
    label2id = {"O": 0}
    # 固有表現タイプのsetを獲得して並び替える
    entity_types = set(
        [e["type"] for entities in entities_list for e in entities]
    )
    entity_types = sorted(entity_types)
    for i, entity_type in enumerate(entity_types):
        # "B-"のIDには奇数番号を割り当てる
        label2id[f"B-{entity_type}"] = i * 2 + 1
        # "I-"のIDには偶数番号を割り当てる
        label2id[f"I-{entity_type}"] = i * 2 + 2
    return label2id

# ラベルとIDを紐づけるdictを作成する
label2id = create_label2id(dataset["train"]["entities"])
id2label = {v:k for k, v in label2id.items()}
pprint(id2label)

{0: 'O',
 1: 'B-その他の組織名',
 2: 'I-その他の組織名',
 3: 'B-イベント名',
 4: 'I-イベント名',
 5: 'B-人名',
 6: 'I-人名',
 7: 'B-地名',
 8: 'I-地名',
 9: 'B-政治的組織名',
 10: 'I-政治的組織名',
 11: 'B-施設名',
 12: 'I-施設名',
 13: 'B-法人名',
 14: 'I-法人名',
 15: 'B-製品名',
 16: 'I-製品名'}


#### データの前処理

In [24]:
from transformers.tokenization_utils_base import BatchEncoding

def preprocess_data(
    data: dict[str, Any],
    tokenizer: PreTrainedTokenizer,
    label2id: dict[int, str],
) -> BatchEncoding:
    """データの前処理"""
    # テキストのトークナイゼーションを行う
    inputs = tokenizer(
        data["text"],
        return_tensors="pt",
        return_special_tokens_mask=True,
    )
    inputs = {k: v.squeeze(0) for k, v in inputs.items()}
    
    # 文字のlistとトークンのlistのアライメントをとる
    characters = list(data["text"])
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"])
    char_to_token_indices, _ = get_alignments(characters, tokens)
    
    # "O"のIDのlistを作成する
    labels = torch.zeros_like(inputs["input_ids"])
    for entity in data["entities"]: # 各固有表現を処理する
        start_token_indices = char_to_token_indices[entity["span"][0]]
        end_token_indices = char_to_token_indices[
            entity["span"][1] - 1
        ]
        # 文字に対するトークンが存在しなければスキップする
        if (
            len(start_token_indices)==0
            or len(end_token_indices)==0
        ):
            continue
        start, end = start_token_indices[0], end_token_indices[0]
        entity_type = entity["type"]
        # 固有表現の開始トークンの位置に"B-"のIDを設定する
        labels[start] = label2id[f"B-{entity_type}"]
        # 固有表現の開始トークン以外の位置に"I-"のIDを設定する
        if start != end:
            labels[start + 1: end + 1] = label2id[f"I-{entity_type}"]
    # 特殊トークンの位置のIDは-100とする
    labels[torch.where(inputs["special_tokens_mask"])] = -100
    inputs["labels"] = labels
    return inputs
    
# 訓練セットに対して前処理を行う
train_dataset = dataset["train"].map(
    preprocess_data,
    fn_kwargs={
        "tokenizer": tokenizer,
        "label2id": label2id,
    },
    remove_columns=dataset["train"].column_names,
)
# 検証セットに対して前処理を行う
validation_dataset = dataset["validation"].map(
    preprocess_data,
    fn_kwargs={
        "tokenizer": tokenizer,
        "label2id": label2id,
    },
    remove_columns=dataset["validation"].column_names,
)

100%|██████████| 534/534 [00:00<00:00, 1580.32ex/s]


#### モデルの準備

In [25]:
from transformers import (
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
)

# モデルを読み込む
model = AutoModelForTokenClassification.from_pretrained(
    model_name, label2id=label2id, id2label=id2label
)
# パラメータをメモリ上に隣接する形で配置
for param in model.parameters():
    param.data = param.data.contiguous()
# collate関数にDataCollatorForTokenClassificationを用いる
data_collator = DataCollatorForTokenClassification(tokenizer)

Some weights of the model checkpoint at tohoku-nlp/bert-base-japanese-v3 were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initi

#### モデルのファインチューニング

In [26]:
from transformers import Trainer, TrainingArguments
from transformers.trainer_utils import set_seed

# 乱数シードを42に固定する
set_seed(42)

# Trainerに渡す引数を初期化する
training_args = TrainingArguments(
    output_dir="../model/output_bert_ner",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=1e-4,
    lr_scheduler_type="linear",
    warmup_ratio=0.1,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    fp16=True,
    report_to="none",
)

# Trainerを初期化する
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
    args=training_args,
)

# 訓練する
trainer.train()

Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4274
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 670


Epoch,Training Loss,Validation Loss
1,0.656300,0.107465
2,0.067700,0.084497
3,0.028600,0.090706
4,0.012100,0.096854
5,0.005500,0.100623


The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 534
  Batch size = 32
Saving model checkpoint to ../model/output_bert_ner/checkpoint-134
Configuration saved in ../model/output_bert_ner/checkpoint-134/config.json
Model weights saved in ../model/output_bert_ner/checkpoint-134/pytorch_model.bin
tokenizer config file saved in ../model/output_bert_ner/checkpoint-134/tokenizer_config.json
Special tokens file saved in ../model/output_bert_ner/checkpoint-134/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForTokenClassification.forw

TrainOutput(global_step=670, training_loss=0.15402125137955394, metrics={'train_runtime': 105.305, 'train_samples_per_second': 202.934, 'train_steps_per_second': 6.362, 'total_flos': 1054773477784752.0, 'train_loss': 0.15402125137955394, 'epoch': 5.0})

### 固有表現の予測・抽出

#### 固有表現ラベルの予測

In [27]:
def convert_list_dict_to_dict_list(
    list_dict: dict[str, list]
) -> list[dict[str, list]]:
    """ミニバッチのデータを事例単位のlistに変換"""
    dict_list = []
    # dictのキーのlistを作成する
    keys = list(list_dict.keys())
    for idx in range(len(list_dict[keys[0]])): # 各事例で処理する
        # dictの各キーからデータを取り出してlistに追加する
        dict_list.append({key: list_dict[key][idx] for key in keys})
    return dict_list

# ミニバッチのデータを事例単位のlistに変換する
list_dict = {
    "input_ids": [[0, 1], [2, 3]],
    "labels": [[1, 2], [3, 4]],
}
dict_list = convert_list_dict_to_dict_list(list_dict)
print(f"入力: {list_dict}")
print(f"出力: {dict_list}")

入力: {'input_ids': [[0, 1], [2, 3]], 'labels': [[1, 2], [3, 4]]}
出力: [{'input_ids': [0, 1], 'labels': [1, 2]}, {'input_ids': [2, 3], 'labels': [3, 4]}]


In [28]:
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import PreTrainedModel

def run_prediction(
    dataloader: DataLoader, 
    model: PreTrainedModel
) -> list[dict[str, Any]]:
    """予測スコアに基づき固有表現ラベルを予測"""
    predictions = []
    for batch in tqdm(dataloader): # 各ミニバッチを処理する
        inputs = {
            k: v.to(model.device)
            for k, v in batch.items()
            if k != "special_tokens_mask"
        }
        # 予測スコアを取得する
        logits = model(**inputs).logits
        # 最もスコアの高いIDを取得する
        batch["pred_label_ids"] = logits.argmax(-1)
        batch = {k: v.cpu().tolist() for k, v in batch.items()}
        # ミニバッチのデータを事例単位のlistに変換する
        predictions += convert_list_dict_to_dict_list(batch)
    return predictions

# ミニバッチの作成にDataLoaderを用いる
validation_dataloader = DataLoader(
    validation_dataset,
    batch_size=32,
    shuffle=False,
    collate_fn=data_collator,
)
# 固有表現ラベルを予測する
predictions = run_prediction(validation_dataloader, model)
print(predictions[0]["pred_label_ids"])

100%|██████████| 17/17 [00:00<00:00, 25.91it/s]

[0, 0, 15, 16, 0, 0, 13, 14, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 14, 14, 14, 13, 0, 0, 0, 0, 0, 15, 15, 16, 16, 13, 14, 14, 14, 16, 0, 13, 13, 14, 14, 13, 0, 0, 0, 13, 14, 14, 14, 0, 0, 13, 14, 14, 14, 0, 0, 0, 0, 0, 15, 16, 16, 0, 13, 14, 14, 14, 14, 0, 0, 0, 0, 15, 15, 16, 0, 13, 13, 14]


#### 固有表現の抽出

In [29]:
from seqeval.metrics.sequence_labeling import get_entities

def extract_entities(
    predictions: list[dict[str, Any]],
    dataset: list[dict[str, Any]],
    tokenizer: PreTrainedTokenizer,
    id2label: dict[int, str],
) -> list[dict[str, Any]]:
    """固有表現を抽出"""
    results = []
    for prediction, data in zip(predictions, dataset):
        # 文字列のlistを取得する
        characters = list(data["text"])
        
        # 特殊トークンを除いたトークンのlistと予測ラベルのlistを取得する
        tokens, pred_labels = [], []
        all_tokens = tokenizer.convert_ids_to_tokens(
            prediction["input_ids"]
        )
        for token, label_id in zip(
            all_tokens, prediction["pred_label_ids"]
        ):
            # 特殊トークン以外をlistに追加する
            if token not in tokenizer.all_special_tokens:
                tokens.append(token)
                pred_labels.append(id2label[label_id])
                
        # 文字のlistとトークンのlistのアライメントを取る
        _, token_to_char_indices = get_alignments(characters, tokens)
        
        # 予測ラベルのlistから固有表現タイプと、
        # トークン単位の開始位置と終了位置を取得して、
        # それらを正解データと同じ形式にする
        pred_entities = []
        for entity in get_entities(pred_labels):
            entity_type, token_start, token_end = entity
            # 文字単位の開始位置を取得する
            char_start = token_to_char_indices[token_start][0]
            # 文字単位の終了位置を取得する
            char_end = token_to_char_indices[token_end][-1] + 1
            pred_entity = {
                "name": "".join(characters[char_start:char_end]),
                "span": [char_start, char_end],
                "type": entity_type,
            }
            pred_entities.append(pred_entity)
        data["pred_entities"] = pred_entities
        results.append(data)
    return results

# 固有表現を抽出する
results = extract_entities(
    predictions, dataset["validation"], tokenizer, id2label
)
pprint(results[0])

{'curid': '1662110',
 'entities': [{'name': '復活篇', 'span': [1, 4], 'type': '製品名'},
              {'name': 'グリーンバニー', 'span': [6, 13], 'type': '法人名'}],
 'pred_entities': [{'name': '復活篇', 'span': [1, 4], 'type': '製品名'},
                   {'name': 'グリーンバニー', 'span': [6, 13], 'type': '法人名'}],
 'text': '「復活篇」はグリーンバニーからの発売となっている。'}


### 検証セットを使ったモデルの選択

In [30]:
from glob import glob

best_score = 0
# 各チェックポイントで処理
for checkpoint in sorted(glob("../model/output_bert_ner/checkpoint-*")):
    # モデルを読み込む
    model = AutoModelForTokenClassification.from_pretrained(
        checkpoint
    )
    model.to("cuda:0") # モデルをGPUに移動
    predictions = run_prediction(validation_dataloader, model)
    # 固有表現を抽出
    results = extract_entities(
        predictions, dataset["validation"], tokenizer, id2label
    )
    # 正解データと予測データのラベルのlistを作成
    true_labels, pred_labels = convert_results_to_labels(results)
    # 評価スコアを算出
    scores = compute_scores(true_labels, pred_labels, "micro")
    if best_score < scores["f1-score"]:
        best_score = scores["f1-score"]
        best_model = model

loading configuration file ../model/output_bert_ner/checkpoint-134/config.json
Model config BertConfig {
  "_name_or_path": "../model/output_bert_ner/checkpoint-134",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-\u305d\u306e\u4ed6\u306e\u7d44\u7e54\u540d",
    "2": "I-\u305d\u306e\u4ed6\u306e\u7d44\u7e54\u540d",
    "3": "B-\u30a4\u30d9\u30f3\u30c8\u540d",
    "4": "I-\u30a4\u30d9\u30f3\u30c8\u540d",
    "5": "B-\u4eba\u540d",
    "6": "I-\u4eba\u540d",
    "7": "B-\u5730\u540d",
    "8": "I-\u5730\u540d",
    "9": "B-\u653f\u6cbb\u7684\u7d44\u7e54\u540d",
    "10": "I-\u653f\u6cbb\u7684\u7d44\u7e54\u540d",
    "11": "B-\u65bd\u8a2d\u540d",
    "12": "I-\u65bd\u8a2d\u540d",
    "13": "B-\u6cd5\u4eba\u540d",
    "14": "I-\u6cd5\u4eba\u540d",
    "15": "B-\u88fd\u54c1\u540d",
    "16": "I-\

### 性能評価

In [32]:
# テストセットに対して前処理
test_dataset = dataset["test"].map(
    preprocess_data,
    fn_kwargs={
        "tokenizer": tokenizer,
        "label2id": label2id,
    },
    remove_columns=dataset["test"].column_names,
)
# ミニバッチ作成にDataLoaderを用いる
test_dataloader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    collate_fn=data_collator,
)
# 固有表現ラベルを予測
predictions = run_prediction(test_dataloader, best_model)
# 固有表現を抽出する
results = extract_entities(
    predictions, dataset["test"], tokenizer, id2label
)
# 正解データと予測データのラベルのlistを作成
true_labels, pred_labels = convert_results_to_labels(results)
# 評価結果を出力
print(classification_report(true_labels, pred_labels))

Loading cached processed dataset at /root/.cache/huggingface/datasets/llm-book___ner-wikipedia-dataset/default/0.0.0/184bcf9be66116e777f2f534436226d47348676c93ba20cca58933f1b2b3b782/cache-108c8ca9ee1f2f9c.arrow
100%|██████████| 17/17 [00:00<00:00, 21.55it/s]


              precision    recall  f1-score   support

     その他の組織名       0.80      0.80      0.80       100
       イベント名       0.81      0.91      0.86        93
          人名       0.95      0.98      0.97       287
          地名       0.85      0.88      0.86       204
      政治的組織名       0.73      0.84      0.78       106
         施設名       0.87      0.85      0.86       137
         法人名       0.86      0.88      0.87       248
         製品名       0.78      0.83      0.80       158

   micro avg       0.85      0.88      0.87      1333
   macro avg       0.83      0.87      0.85      1333
weighted avg       0.85      0.88      0.87      1333



### エラー分析

In [38]:
def find_error_results(
    results: list[dict[str, Any]],
) -> list[dict[str, Any]]:
    """エラー事例を発見"""
    error_results = []
    for idx, result in enumerate(results): # 各事例を処理
        result["idx"] = idx
        if result["entities"] != result["pred_entities"]:
            error_results.append(result)
    return error_results

def output_text_with_label(result: dict[str, Any], entity_column: str) -> str:
    """固有表現ラベル付きテキストを出力"""
    text_with_label = ""
    entity_count = 0
    for i, char in enumerate(result["text"]): # 各文字を処理
        # 出力に加えていない固有表現の有無を判定
        if entity_count < len(result[entity_column]):
            entity = result[entity_column][entity_count]
            # 固有表現の先頭の処理
            if i == entity["span"][0]:
                entity_type = entity["type"]
                text_with_label += f"[({entity_type})]"
            text_with_label += char
            # 固有表現の末尾の処理
            if i == entity["span"][1] - 1:
                text_with_label += "]"
                entity_count += 1
        else:
            text_with_label += char
    return text_with_label

# エラー事例を発見する
error_results = find_error_results(results)
# 3件のエラー事例を出力する
for result in error_results[:3]:
    idx = result["idx"]
    true_text = output_text_with_label(result, "entities")
    pred_text = output_text_with_label(result, "pred_entities")
    print(f"事例{idx}の正解{true_text}")
    print(f"事例{idx}の予測{pred_text}")
    print()
        

事例16の正解「[(製品名)]兵動・野爆のキャンピング王国]」とのコラボレーション番組。
事例16の予測「[(人名)]兵動・野爆][(製品名)]のキャンピング王国]」とのコラボレーション番組。

事例18の正解[(法人名)]常盤木学園]時代の同級生に[(その他の組織名)]なでしこジャパン]の[(人名)]熊谷紗希]がいる。
事例18の予測[(施設名)]常盤木学園]時代の同級生に[(その他の組織名)]なでしこジャパン]の[(人名)]熊谷紗希]がいる。

事例19の正解テレビで狼男映画の「[(製品名)]倫敦の人狼]」を見た[(人名)]フィル・エヴァリー]は「ロンドンの狼男というタイトルで踊り騒げる曲を書いてみないか」と[(法人名)]ジヴォン]に持ちかけた。
事例19の予測テレビで狼男映画の「[(製品名)]倫敦の人狼]」を見た[(人名)]フィル・エヴァリー]は「[(製品名)]ロンドンの狼男]というタイトルで踊り騒げる曲を書いてみないか」と[(人名)]ジヴォン]に持ちかけた。



### ラベル間の遷移可能性を考慮した予測

### 遷移スコアを定義

In [39]:
def create_transitions(
    label2id: dict[str, int]
) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """遷移スコアを定義"""
    # "B-"のラベルIDのlist
    b_ids = [v for k,v in label2id.items() if k[0]=="B"]
    # "I-"のラベルIDのlist
    i_ids = [v for k,v in label2id.items() if k[0]=="I"]
    # "O"のラベルIDのlist
    o_id = label2id["O"]
    
    # 開始遷移スコアを定義
    # すべてのスコアを-100で初期化
    start_transitions = torch.full([len(label2id)], -100.0)
    # "B-"のラベルへの遷移可能として0を代入する
    start_transitions[b_ids] = 0
    # "O-"のラベルへ遷移可能として0を代入する
    start_transitions[o_id] = 0
    
    # ラベル間の遷移スコアを定義
    # すべてのスコアを-100で初期化
    transitions = torch.full([len(label2id), len(label2id)], -100.0)
    # すべてのラベルから"B-"へ遷移可能として0を代入する
    transitions[:, b_ids] = 0
    # すべてのラベルから"O-"へ遷移可能として0を代入する
    transitions[:, o_id] = 0
    # "B-"から同じタイプの"I-"へ遷移可能として0を代入する
    transitions[b_ids, i_ids] = 0
    # "I-"から同じタイプの"I-"へ遷移可能として0を代入する
    transitions[i_ids, i_ids] = 0
    
    # 終了遷移スコアを定義する
    # すべてのラベルから遷移可能としてすべてのスコアを0とする
    end_transitions = torch.zeros(len(label2id))
    return start_transitions, transitions, end_transitions

# 遷移スコアを定義する
start_transitions, transitions, end_transitions = create_transitions(
    label2id
)

In [40]:
start_transitions

tensor([   0.,    0., -100.,    0., -100.,    0., -100.,    0., -100.,    0.,
        -100.,    0., -100.,    0., -100.,    0., -100.])

### ビタビアルゴリズムを用いたラベル列の予測

In [43]:
def decode_with_viterbi(
    emissions: torch.Tensor, # ラベルの予測スコア
    mask: torch.Tensor, # マスク
    start_transitions: torch.Tensor, # 開始遷移スコア
    transitions: torch.Tensor, # ラベル間の遷移スコア
    end_transitions: torch.Tensor, # 終了遷移スコア
) -> torch.Tensor:
    """ビタビアルゴリズムを用いて最適なラベル列を探索"""
    # バッチサイズと系列長を取得
    batch_size, seq_length = mask.shape
    # 予測スコアとマスクに関して、0次元目と1次元目を入れ替える
    emissions = emissions.transpose(1, 0)
    mask = mask.transpose(1, 0)
    
    histories = [] # 最適なラベル系列を保持するための履歴list
    # 開始遷移スコアと予測スコアを加算して、累積スコアの初期値とする
    score = start_transitions + emissions[0]
    for i in range(1, seq_length):
        # 累積スコアを3次元に変換
        broadcast_score = score.unsqueeze(2)
        # 現在の予測スコアを3次元に変換
        broadcast_emission = emissions[i].unsqueeze(1)
        # 累積スコアと遷移スコアと現在の予測スコアを加算して、
        # 現在の累積スコアを取得する
        next_score = (
            broadcast_score + transitions + broadcast_emission
        )
        # 現在の累積スコアの各ラベルの最大値とそのインデックスを取得する
        next_score, indices = next_score.max(dim=1)
        # マスクしない要素の場合、累積スコアを更新する
        score = torch.where(mask[i].unsqueeze(1), next_score, score)
        # スコアの高いインデックスを履歴のlistに追加する
        histories.append(indices)
    # 終了遷移スコアを加算して合計スコアとする
    score += end_transitions
    
    # 各事例で最適なラベル列を取得する
    best_labels_list = []
    for i in range(batch_size):
        # 合計スコアの中で最大のスコアとなるラベルを取得する
        _, best_last_label = score[i].max(dim=0)
        best_labels = [best_last_label.item()]
        # 最後のラベルの遷移を逆方向に探索し、最適なラベル列を取得する
        for history in reversed(histories):
            best_last_label = history[i][best_labels[-1]]
            best_labels.append(best_last_label.item())
        # 順序を反転する
        best_labels.reverse()
        best_labels_list.append(best_labels)
    return torch.LongTensor(best_labels_list)

In [44]:
def run_prediction_viterbi(
    dataloader: DataLoader,
    model: PreTrainedModel,
) -> list[dict[str, Any]]:
    """ビタビアルゴリズムを用いてラベルを予測"""
    # 遷移スコアを取得する
    start_transitions, transitions, end_transitions = create_transitions(
        model.config.label2id
    )
    
    predictions = []
    for batch in tqdm(dataloader):
        inputs = {
            k: v.to(model.device)
            for k,v in batch.items()
            if k != "special_tokens_mask"
        }
        # [CLS]以外の予測スコアを取得する
        logits = model(**inputs).logits.cpu()[:, 1:, :]
        # [CLS]以外の特殊トークンのマスクを取得する
        mask = (batch["special_tokens_mask"].cpu() == 0)[:, 1:]
        # ビタビアルゴリズムを用いて最適なIDの系列を探索する
        pred_label_ids = decode_with_viterbi(
            logits,
            mask,
            start_transitions,
            transitions,
            end_transitions,
        )
        # [CLS]のIDを0とする
        cls_pred_label_id = torch.zeros(pred_label_ids.shape[0], 1)
        # [CLS]のIDと探索したIDの系列を連結して予測ラベルとする
        batch["pred_label_ids"] = torch.concat(
            [cls_pred_label_id, pred_label_ids], dim=1
        )
        batch = {k:v.cpu().tolist() for k,v in batch.items()}
        # ミニバッチのデータを事例単位のlistに変換する
        predictions += convert_list_dict_to_dict_list(batch)
    return predictions

# ビタビアルゴリズムを用いてラベルを予測する
predictions = run_prediction_viterbi(test_dataloader, best_model)
# 固有表現を抽出する
results = extract_entities(
    predictions, dataset["test"], tokenizer, id2label
)
# 正解データと予測データのラベルのlistを作成する
true_labels, pred_labels = convert_results_to_labels(results)
# 評価結果を出力する
print(classification_report(true_labels, pred_labels))

100%|██████████| 17/17 [00:01<00:00, 14.57it/s]


              precision    recall  f1-score   support

     その他の組織名       0.82      0.80      0.81       100
       イベント名       0.88      0.92      0.90        93
          人名       0.95      0.97      0.96       287
          地名       0.87      0.89      0.88       204
      政治的組織名       0.77      0.87      0.82       106
         施設名       0.92      0.86      0.89       137
         法人名       0.89      0.88      0.89       248
         製品名       0.80      0.84      0.82       158

   micro avg       0.88      0.89      0.88      1333
   macro avg       0.86      0.88      0.87      1333
weighted avg       0.88      0.89      0.88      1333



In [45]:
idx = 27
result = results[idx]
true_text = output_text_with_label(result, "entities")
pred_text = output_text_with_label(result, "pred_entities")
print(f"事例{idx}の正解: {true_text}")
print(f"事例{idx}の予測: {pred_text}")

事例27の正解: [(政治的組織名)]李承晩政権]期から[(政治的組織名)]朴正煕政権]期の1970年前後まで、南側の[(地名)]大韓民国]よりも北側の[(地名)]朝鮮民主主義人民共和国]の方が経済的な体力では勝っていたのである。
事例27の予測: [(人名)]李承晩政権]期から[(人名)]朴正煕政権]期の1970年前後まで、南側の[(地名)]大韓民国]よりも北側の[(地名)]朝鮮民主主義人民共和国]の方が経済的な体力では勝っていたのである。


### CRFによるラベル間の遷移可能性の学習

#### BERT-CRFモデルの実装

In [47]:
# !pip install pytorch-crf

In [48]:
from torchcrf import CRF
from transformers import BertForTokenClassification, PretrainedConfig
from transformers.modeling_outputs import TokenClassifierOutput

class BertWithCrfTokenClassification(BertForTokenClassification):
    """BertForTokenClassificationにCRF層を加えたクラス"""
    
    def __init__(self, config:PretrainedConfig):
        """クラスの初期化"""
        super().__init__(config)
        # CRF層を追加する
        self.crf = CRF(len(config.label2id), batch_first=True)
        
    def _init_weights(self, module: torch.nn.Module) -> None:
        """定義した遷移スコアでパラメータ初期化"""
        super()._init_weights(module)
        if isinstance(module, CRF):
            st, t, et = create_transitions(self.config.label2id)
            module.start_transitions.data = st
            module.transitions.data = t
            module.end_transitions.data = et
            
    def forward(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor | None = None,
        token_type_ids: torch.Tensor | None = None,
        labels: torch.Tensor | None = None,    
    ) -> TokenClassifierOutput:
        """モデルの前向き計算を定義"""
        # BertForTokenClassificationのforwardメソッドを適用して、
        # 予測スコアを算出する
        output = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        if labels is not None:
            logits = output.logits
            mask = labels != -100
            labels *= mask
            # CRFによる損失を計算
            output["loss"] = -self.crf(
                logits[:, 1:, :],
                labels[:, 1:],
                mask=mask[:, 1:],
                reduction="mean",
            )
        return output
    
# BertForTokenClassificationにCRF層を加えたクラスを定義
model_crf = BertWithCrfTokenClassification.from_pretrained(
    model_name, label2id=label2id, id2label=id2label
)

loading configuration file https://huggingface.co/tohoku-nlp/bert-base-japanese-v3/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/2c9ec44dde23b6b02ec9f34997dd1595677490da33cc6f7f5931a82fea500ef1.fa452780f4f534fd5a9a500fd6dc0ab2b41b7f3a87fde31e4e439dacbbe6eea3
Model config BertConfig {
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-\u305d\u306e\u4ed6\u306e\u7d44\u7e54\u540d",
    "2": "I-\u305d\u306e\u4ed6\u306e\u7d44\u7e54\u540d",
    "3": "B-\u30a4\u30d9\u30f3\u30c8\u540d",
    "4": "I-\u30a4\u30d9\u30f3\u30c8\u540d",
    "5": "B-\u4eba\u540d",
    "6": "I-\u4eba\u540d",
    "7": "B-\u5730\u540d",
    "8": "I-\u5730\u540d",
    "9": "B-\u653f\u6cbb\u7684\u7d44\u7e54\u540d",
    "10": "I-\u653f\u6cbb\u7684\u7d44\u7e54\u540d",
    "11": "B-\u65bd\u8a2d\u540d",
    "12":

In [51]:
# 乱数シードを再設定
set_seed(42)

# Trainerに渡す引数を初期化する
training_args = TrainingArguments(
    output_dir="../model/output_bert_crf_ner",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=1e-4,
    lr_scheduler_type="linear",
    warmup_ratio=0.1,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    fp16=True,
)

# Trainerを初期化
trainer = Trainer(
    model=model_crf,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
    args=training_args,
)

# 学習
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `BertWithCrfTokenClassification.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertWithCrfTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4274
  Num Epochs = 5
 

Epoch,Training Loss,Validation Loss
1,17.101500,1.570267
2,1.221400,1.424493
3,0.587100,1.503783
4,0.306400,1.747114
5,0.187800,1.922975


The following columns in the evaluation set don't have a corresponding argument in `BertWithCrfTokenClassification.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertWithCrfTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 534
  Batch size = 32
Saving model checkpoint to ../model/output_bert_crf_ner/checkpoint-134
Configuration saved in ../model/output_bert_crf_ner/checkpoint-134/config.json
Model weights saved in ../model/output_bert_crf_ner/checkpoint-134/pytorch_model.bin
tokenizer config file saved in ../model/output_bert_crf_ner/checkpoint-134/tokenizer_config.json
Special tokens file saved in ../model/output_bert_crf_ner/checkpoint-134/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertWithCrfTokenClassification.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by 

TrainOutput(global_step=670, training_loss=3.8808603286743164, metrics={'train_runtime': 192.622, 'train_samples_per_second': 110.943, 'train_steps_per_second': 3.478, 'total_flos': 1054777482669504.0, 'train_loss': 3.8808603286743164, 'epoch': 5.0})

In [58]:
def run_prediction_crf(
    dataloader: DataLoader,
    model: PreTrainedModel,
) -> list[dict[str, Any]]:
    """BERT-CRFモデルを用いてラベルを予測"""
    predictions = []
    for batch in tqdm(dataloader):
        inputs = {
            k:v.to(model.device)
            for k,v in batch.items()
            if k != "special_tokens_mask"
        }
        # [CLS]以外の予測スコアを取得
        logits = model(**inputs).logits.cpu()[:, 1:, :]
        # [CLS]以外の特殊トークンのマスクを取得する
        mask = (batch["special_tokens_mask"]==0).cpu()[:, 1:]
        # 訓練した遷移スコアを取得する
        start_transitions = model_crf.crf.start_transitions.cpu()
        transitions = model_crf.crf.transitions.cpu()
        end_transitions = model_crf.crf.end_transitions.cpu()
        # ビタビアルゴリズムを用いて最適なIDの系列を探索する
        pred_label_ids = decode_with_viterbi(
            logits,
            mask,
            start_transitions,
            transitions,
            end_transitions
        )
        # [CLS]のIDを0とする
        cls_pred_label_id = torch.zeros(pred_label_ids.shape[0], 1)
        # [CLS]のIDと探索したIDの系列を連結して予測ラベルとする
        batch["pred_label_ids"] = torch.concat(
            [cls_pred_label_id, pred_label_ids], dim=1
        )
        batch = {k: v.cpu().tolist() for k,v in batch.items()}
        # ミニバッチのデータを事例単位のlistに変換
        predictions += convert_list_dict_to_dict_list(batch)
    return predictions

In [59]:
# Checkpointからモデル選択
best_score = 0
for checkpoint in sorted(glob("../model/output_bert_crf_ner/checkpoint-*")):
    # モデル読み込み
    model_crf = BertWithCrfTokenClassification.from_pretrained(
        checkpoint
    )
    model_crf = model_crf.to("cuda:0")
    # 固有表現ラベルを予測
    predictions = run_prediction_crf(validation_dataloader, model_crf)
    # 固有表現を抽出
    results = extract_entities(
        predictions, dataset["validation"], tokenizer, id2label
    )
    # 正解データと予測データのラベルのlistを作成
    true_labels, pred_labels = convert_results_to_labels(results)
    # 評価スコアを算出
    socres = compute_scores(true_labels, pred_labels, "micro")
    if best_score < socres["f1-score"]:
        best_score = scores["f1-score"]
        best_model_crf = model_crf

loading configuration file ../model/output_bert_crf_ner/checkpoint-134/config.json
Model config BertConfig {
  "_name_or_path": "tohoku-nlp/bert-base-japanese-v3",
  "architectures": [
    "BertWithCrfTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-\u305d\u306e\u4ed6\u306e\u7d44\u7e54\u540d",
    "2": "I-\u305d\u306e\u4ed6\u306e\u7d44\u7e54\u540d",
    "3": "B-\u30a4\u30d9\u30f3\u30c8\u540d",
    "4": "I-\u30a4\u30d9\u30f3\u30c8\u540d",
    "5": "B-\u4eba\u540d",
    "6": "I-\u4eba\u540d",
    "7": "B-\u5730\u540d",
    "8": "I-\u5730\u540d",
    "9": "B-\u653f\u6cbb\u7684\u7d44\u7e54\u540d",
    "10": "I-\u653f\u6cbb\u7684\u7d44\u7e54\u540d",
    "11": "B-\u65bd\u8a2d\u540d",
    "12": "I-\u65bd\u8a2d\u540d",
    "13": "B-\u6cd5\u4eba\u540d",
    "14": "I-\u6cd5\u4eba\u540d",
    "15": "B-\u88fd\u54c1\u540d",
    "16": "I-

In [60]:
# 固有表現ラベルを予測する
predictions = run_prediction_crf(test_dataloader, best_model_crf)
# 固有表現を抽出
results = extract_entities(
    predictions, dataset["test"], tokenizer, id2label
)
# 正解データと予測データのラベルのlistを作成
true_labels, pred_labels = convert_results_to_labels(results)
# 評価結果
print(classification_report(true_labels, pred_labels))

100%|██████████| 17/17 [00:01<00:00,  8.88it/s]


              precision    recall  f1-score   support

     その他の組織名       0.83      0.82      0.82       100
       イベント名       0.88      0.95      0.91        93
          人名       0.95      0.96      0.96       287
          地名       0.88      0.87      0.87       204
      政治的組織名       0.83      0.89      0.86       106
         施設名       0.90      0.87      0.88       137
         法人名       0.89      0.87      0.88       248
         製品名       0.80      0.85      0.82       158

   micro avg       0.88      0.89      0.89      1333
   macro avg       0.87      0.88      0.88      1333
weighted avg       0.88      0.89      0.89      1333

